In [6]:
print('hello world')

hello world


In [9]:
import requests
from bs4 import BeautifulSoup
url = "https://www.ptt.cc/bbs/joke/index.html"
for i in range(3): #往上爬3頁
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    sel = soup.select("div.title a") #標題
    u = soup.select("div.btn-group.btn-group-paging a") #a標籤
    print ("本頁的URL為"+url)
    url = "https://www.ptt.cc"+ u[1]["href"] #上一頁的網址

for s in sel: #印出網址跟標題
    print(s["href"],s.text)

本頁的URL為https://www.ptt.cc/bbs/joke/index.html
本頁的URL為https://www.ptt.cc/bbs/joke/index8657.html
本頁的URL為https://www.ptt.cc/bbs/joke/index8656.html
/bbs/joke/M.1744984695.A.F9B.html [猜謎] 哪個比較痛？
/bbs/joke/M.1745030487.A.E3F.html [耍冷] 沒事多喝水
/bbs/joke/M.1745034375.A.2F9.html [耍冷] 推特上在夯什麼 Part.1852
/bbs/joke/M.1745035978.A.848.html [趣圖] 你爸真的是MVP
/bbs/joke/M.1745037601.A.49E.html [ＸＤ] 奉勸各位要學好物理
/bbs/joke/M.1745060690.A.AA7.html [耍冷] 這世界上
/bbs/joke/M.1745065272.A.1A3.html [趣事] 眼鏡蛇攻擊直升機
/bbs/joke/M.1745066426.A.601.html [ＸＤ] 台派
/bbs/joke/M.1745119104.A.963.html [耍冷] where is Sue?
/bbs/joke/M.1745124181.A.6A7.html [耍冷] 推特上在夯什麼 Part.1853
/bbs/joke/M.1745130916.A.9AD.html [ＸＤ] 鬼壓床怎麼辦
/bbs/joke/M.1745152175.A.276.html Re: [耍冷] 為什麼很多人覺得穿CK內衣很性感？
/bbs/joke/M.1745158801.A.121.html [趣圖] 寵物進去出來會變兩隻
/bbs/joke/M.1745159150.A.4F6.html [地獄] 蕭煌奇在瞎雞巴煮什麼料理
/bbs/joke/M.1745159552.A.97D.html [地獄] 東方聖人與西方聖人，誰比較強？
/bbs/joke/M.1745188485.A.CF8.html Re: [ＸＤ] 鬼壓床怎麼辦
/bbs/joke/M.1745190057.A.580.html [地獄] 玻璃睪丸在古代的叫什麼？

In [10]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, parse_qs, urlunparse
from markdownify import markdownify as md
from datetime import datetime
import json

visited = set()
all_data = []
link_map = {}  # 新增：記錄每頁的 outgoing link
MAX_VISIT = 10

def clean_url(url):
    parsed = urlparse(url)
    query = parse_qs(parsed.query)
    whitelist = []  # 可自訂保留的參數
    filtered_query = {k: v for k, v in query.items() if k in whitelist}
    new_query = "&".join([f"{k}={v[0]}" for k, v in filtered_query.items()])
    clean_parsed = parsed._replace(query=new_query)
    return urlunparse(clean_parsed)

def crawl(url):
    try:
        t0 = datetime.now()
        res = requests.get(url, timeout=5)
        res.encoding = res.apparent_encoding
        delay = (datetime.now() - t0).total_seconds()

        soup = BeautifulSoup(res.text, "html.parser")
        title = soup.title.string if soup.title else ""
        description = soup.find("meta", attrs={"name": "description"})
        desc = description["content"] if description else ""
        markdown = md(res.text)

        links = set()
        for a in soup.find_all("a", href=True):
            full_url = urljoin(url, a['href'])
            cleaned = clean_url(full_url)
            if cleaned.startswith("http"):
                links.add(cleaned)

        # 儲存這一頁指向的所有 link
        link_map[url] = list(links)

        data = {
            "url": url,
            "domain": urlparse(url).netloc,
            "title": title.strip(),
            "description": desc.strip(),
            "markdown": markdown[:],
            "links": link_map[url],
            "created_time": datetime.utcnow().isoformat(),
            "delay_time": delay,
            "page_rank": 0.0  # 初始為 0，之後會補
        }

        return data, links
    except Exception as e:
        print(f"[ERROR] {url}：{e}")
        return None, set()

def recursive_crawl(url):
    if url in visited:
        print(f"[STOP] Already visited: {url}")
        return
    if len(visited) >= MAX_VISIT:
        print("[STOP] Reached max visit limit.")
        return

    print(f"[CRAWL] Visiting: {url}")
    visited.add(url)

    data, links = crawl(url)
    if data:
        all_data.append(data)
        for link in links:
            recursive_crawl(link)

# 🚀 啟動
seed_url = "https://zh.wikipedia.org"
recursive_crawl(seed_url)

# 🧠 PageRank（簡化版：統計被指向的次數）
page_rank_dict = {}
for src, targets in link_map.items():
    for tgt in targets:
        if tgt in page_rank_dict:
            page_rank_dict[tgt] += 1
        else:
            page_rank_dict[tgt] = 1

# 🪄 回填每一頁的 page_rank 值
for item in all_data:
    item["page_rank"] = page_rank_dict.get(item["url"], 0)

# 💾 儲存結果
with open("recursive_crawl_result.json", "w", encoding="utf-8") as f:
    json.dump(all_data, f, ensure_ascii=False, indent=2)


[CRAWL] Visiting: https://zh.wikipedia.org
[CRAWL] Visiting: https://zh.wikipedia.org/wiki/Special:%E7%BB%9F%E8%AE%A1
[STOP] Already visited: https://zh.wikipedia.org/wiki/Special:%E7%BB%9F%E8%AE%A1
[CRAWL] Visiting: https://zh.wikipedia.org/wiki/Wikipedia:%E5%B7%A1%E6%9F%A5%E5%91%98
[CRAWL] Visiting: https://zh.wikipedia.org/wiki/Template:Copyedit
[STOP] Already visited: https://zh.wikipedia.org/wiki/Template:Copyedit
[CRAWL] Visiting: https://tl.wikipedia.org/wiki/Padron:Linis-kopya
[STOP] Already visited: https://tl.wikipedia.org/wiki/Padron:Linis-kopya
[CRAWL] Visiting: https://developer.wikimedia.org
[CRAWL] Visiting: https://developer.wikimedia.org/zh-hant/
[STOP] Already visited: https://developer.wikimedia.org/zh-hant/
[CRAWL] Visiting: https://developer.wikimedia.org/zh-hant/community/grow/
[STOP] Already visited: https://developer.wikimedia.org/zh-hant/
[STOP] Already visited: https://developer.wikimedia.org/zh-hant/community/grow/
[CRAWL] Visiting: https://developer.wikimedi

In [3]:
import sys 
print(sys.executable)

c:\Users\User\AppData\Local\Programs\Python\Python313\python.exe


In [ ]:
# markdown 不要截斷，要留下完整，上面的URL只有擷取到前500字元 

In [ ]:
# page rank不用算，這些資料之後提交到API，但我還沒設計好